# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import openweathermapy.core as owm 
import os
import urllib

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []
countries = []
keys = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    country_code = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
    # create key for cities with the same name to use in API call (e.g. Athens GR vs Athens GA)
    key = city+', '+country_code 

    # If the city is unique, then add it to a our cities list
    if key not in keys:
        cities.append(city)
        countries.append(country_code)
        keys.append(key)

# Print the city count to confirm sufficient count
print(len(keys))


642


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
weather = []
i=1

try: os.mkdir('output')
except: pass

log = os.path.join("output/log.txt")
with open(log, 'w') as textfile:
    
    settings = {"units": "imperial", "appid": api_key}
    for key in keys:
        try:
            cityweather = owm.get_current(key, **settings)
            textfile.write(f"Processing record {i}: {cityweather['name']}, ID: {cityweather['id']} \n")        
        except urllib.request.HTTPError as e:
            if e.code == 429:
                textfile.write(f"Processing record {i}: {key}, {e}. Retrying in 61 sec\n")
                time.sleep(61)
                cityweather = owm.get_current(key, **settings)
            elif e.code == 404:
                textfile.write(f"{key} not found. Skipping City...\n")
                continue
            else:
                textfile.write(f"{e} Skipping {key}...\n")
                continue
        except BaseException as e:
                textfile.write(f"{e} Skipping {key}...\n")
                continue

        weather += [{'City':cityweather['name'],
                     'Country':cityweather['sys']['country'],
                     'City ID': cityweather['id'],
                     'Latitude':cityweather['coord']['lat'],
                     'Longditude':cityweather['coord']['lon'],
                     'Cloudiness':cityweather['clouds']['all'],
                     'Humidity':cityweather['main']['humidity'],
                     'Current Temp':cityweather['main']['temp'],
                     'Max Temp':cityweather['main']['temp_max'],
                     'Min Temp':cityweather['main']['temp_min'],
                     # take avg to normalize for different time zones 
                     'Avg Temp':(cityweather['main']['temp_max']+cityweather['main']['temp_min'])/2,
                     'Wind Speed':cityweather['wind']['speed']
                    }]
        i += 1

with open(log, 'r') as textfile:                       
    print(textfile.read())
textfile.close()

Processing record 1: Butaritari, ID: 2110227 
Processing record 2: Tahe, ID: 2034615 
Processing record 3: Port Alfred, ID: 964432 
Processing record 4: Santa Quiteria do Maranhao, ID: 3389358 
Processing record 5: Codajas, ID: 3664525 
Processing record 6: Kapaa, ID: 5848280 
Processing record 7: Contramaestre, ID: 3563504 
Processing record 8: Comodoro Rivadavia, ID: 3860443 
Processing record 9: Cape Town, ID: 3369157 
Processing record 10: Ambulu, ID: 1621313 
Processing record 11: Iqaluit, ID: 5983720 
Processing record 12: Lexington, ID: 4297983 
Processing record 13: Aksarka, ID: 1512019 
Processing record 14: Ushuaia, ID: 3833367 
Processing record 15: Albany, ID: 2077963 
Processing record 16: Roros, ID: 3141332 
Processing record 17: Alice Springs, ID: 2077895 
Processing record 18: Bokoro, ID: 2434985 
mataura, pf not found. Skipping City...
Processing record 19: Atuona, ID: 4020109 
Processing record 20: Skjervoy, ID: 777682 
Processing record 21: Katsuura, ID: 1865309 
ang

In [4]:
# Print new city count after lookup to confirm the count remains sufficient (to account for API calls not returned)
len(weather)

579

In [5]:
# Create DataFrame
weather_df = pd.DataFrame(weather)

# Change order of Columns
weather_df = weather_df[['City', 
                         'Country', 
                         'City ID',
                         'Latitude', 
                         'Longditude', 
                         'Cloudiness', 
                         'Humidity', 
                         'Current Temp', 
                         'Max Temp', 
                         'Min Temp', 
                         'Avg Temp', 
                         'Wind Speed']]

# Export the city data into a .csv
weather_df.to_csv("output/weather.csv")

# Display the DataFrame
weather_df.head()

,City,Country,City ID,Latitude,Longditude,Cloudiness,Humidity,Current Temp,Max Temp,Min Temp,Avg Temp,Wind Speed
0,Butaritari,KI,2110227,3.07,172.79,92,100,83.25,83.25,83.25,83.25,19.82
1,Tahe,CN,2034615,52.34,124.71,0,61,-0.05,-0.05,-0.05,-0.05,2.93
2,Port Alfred,ZA,964432,-33.59,26.89,12,93,70.09,71.01,69.01,70.01,11.77
3,Santa Quiteria do Maranhao,BR,3389358,-3.52,-42.55,88,78,82.35,82.35,82.35,82.35,4.72
4,Codajas,BR,3664525,-3.84,-62.06,8,79,84.64,84.64,84.64,84.64,5.39


### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot